In [3]:
import pandas as pd
import sqlite3
import datetime

# Connexion à la base SQLite
conn = sqlite3.connect("fabric_sim.db")

def load_data(conn):
    """
    Charge et fusionne les données contract_basis et absence.
    Calcule le total des jours d'absence et le taux d'absence.
    """
    absence = pd.read_sql("SELECT * FROM absence", conn)
    contract = pd.read_sql("SELECT * FROM contract_basis", conn)

    df = contract.merge(absence, on=["firm_id", "person_id", "department_id", "category_id"], how="left")

    # Identifier les colonnes d'absence par code
    absence_columns = [col for col in df.columns if col.startswith("qty_") and col.endswith("_days")]

    # Nettoyage + conversion
    df[absence_columns] = df[absence_columns].fillna(0)
    for col in absence_columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Calcul total jours d'absence toutes causes
    df['total_absence_days'] = df[absence_columns].sum(axis=1)

    # Calcul taux d'absence
    df['absence_rate'] = df['total_absence_days'] / df['qty_working_days']
    df['absence_rate'] = df['absence_rate'].replace([float('inf'), -float('inf')], None)

    return df

def load_absence_type(conn):
    """
    Charge la table absence_type (référentiel des codes d’absence)
    """
    return pd.read_sql("SELECT * FROM absence_type", conn)

df = load_data(conn)

df.head(10)


,contract_zip_code,firm_id,department_id,category_id,person_id,contract_start_date,contract_end_date,company_start_date,birth_date,contract_terminatio_reason,...,freq_p0_days,freq_p1_days,freq_p2_days,freq_p3_days,freq_a1_days,freq_a2_days,qty_days_worked,qty_working_days,total_absence_days,absence_rate
0,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,21.0,21.0,1.000000
1,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,21.0,30.0,1.428571
2,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,23.0,23.0,1.000000
3,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,21.0,21.0,1.000000
4,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,22.0,22.0,1.000000
5,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,22.0,22.0,1.000000
6,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,21.0,21.0,1.000000
7,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,23.0,23.0,1.000000
8,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,22.0,22.0,1.000000
9,5600,19904,0,1,10270,01/07/2014,15/10/2017,01/07/2014,03/01/1978,Préavis travailleur,...,0.0,0.0,0.0,0.0,0.0,0.0,5,21.0,21.0,1.000000


In [2]:
# === MESURES SÉMANTIQUES ===

def total_employees_by_firm(df):
    return df.groupby("firm_id")["person_id"].nunique().reset_index(name="employee_count")

def average_absence_rate_by_firm(df):
    return df.groupby("firm_id")["absence_rate"].mean().reset_index()

def contract_type_distribution_by_firm(df):
    return df.groupby(["firm_id", "contract_type"])["person_id"].count().reset_index(name="count")

def filter_by_contract_type(df, contract_type="CDI"):
    return df[df["contract_type"] == contract_type]


# === ENRICHISSEMENT AVEC absence_type ===

def enrich_absence_with_type(absence_df, absence_type_df):
    """
    Transforme les colonnes qty_*_days en format long et les relie aux types d'absences.
    """
    qty_columns = [col for col in absence_df.columns if col.startswith("qty_") and col.endswith("_days")]

    melted = absence_df.melt(
        id_vars=["firm_id", "department_id", "category_id", "person_id", "year", "month"],
        value_vars=qty_columns,
        var_name="absence_code_col",
        value_name="days"
    )

    # Extraire le code d'absence (ex: qty_p1_days → P1)
    melted["type_absence_code"] = melted["absence_code_col"].str.extract(r"qty_([a-z0-9]+)_days", expand=False).str.upper()

    # Patch : renommer la colonne pour pouvoir merger
    absence_type_df = absence_type_df.rename(columns={"type_absence": "type_absence_code"})

    # Merge avec référentiel
    enriched = melted.merge(absence_type_df, on="type_absence_code", how="left")

    # Garder uniquement les absences réelles
    enriched = enriched[enriched["days"] > 0]

    return enriched


def absences_by_type(enriched_absences_df):
    """
    Agrège les absences par type pour affichage.
    """
    return enriched_absences_df.groupby("type_absence_fr")["days"].sum().reset_index().sort_values(by="days", ascending=False)

def gender_distribution_by_firm(df):
    return df.groupby(["firm_id", "gender"])["person_id"].nunique().reset_index(name="employee_count")


def average_age_by_firm(df):
    today = pd.Timestamp(datetime.today())
    df['birth_date'] = pd.to_datetime(df['birth_date'], errors='coerce')
    df['age'] = ((today - df['birth_date']).dt.days / 365.25).round(1)
    return df.groupby("firm_id")["age"].mean().reset_index(name="average_age")